In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [63]:
# load train and test dataset
def load_dataset():
    # load dataset
    data_x = np.load('fractal_reshapeRecPlot_data_x_aumentated_normalized.npy')
    data_y = np.load('fractal_reshapeRecPlot_data_y_aumentated_normalized.npy')
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
 # one hot encode target values
#     train_y = to_categorical(train_y)
#     test_y = to_categorical(test_y)
#     validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [64]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [65]:
def to_numerical(y):
    ans = []
    print(y[0])
    for x in y:
        #print(x)
        ans.append(np.argmax(x))
        break
    return ans

In [66]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset()

TOTAL_IMAGES =  1184
train_size =  947
test_size =  118
validation_size =  119


In [105]:
data_x = np.load('fractal_reshapeRecPlot_data_x_aumentated_normalized.npy')
data_y = np.load('fractal_reshapeRecPlot_data_y_aumentated_normalized.npy')
test_x = data_x
test_y = data_y

In [106]:
trainY[2]

3

In [107]:
# trainY = to_numerical(trainY)
# testY = to_numerical(testY)
# validationY = to_numerical(validationY)
#trainX, testX = prep_pixels(trainX, testX)

In [108]:
print('Classes : ', len(set(testY)))

Classes :  4


In [109]:
clf1 = tf.keras.models.load_model("resnet50_100epochs_fractal_augmented_kfold_1.npy")

In [110]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [111]:
clf2 = tf.keras.models.load_model("resnet50_100epochs_fractal_augmented_kfold_2.npy")

In [112]:
clf3 = tf.keras.models.load_model("mobilenet_100epochs_fractal_augmented_kfold_1.npy")

# VGG + Flatting + Gaussian

In [113]:
clf1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          589856    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 64)          1

In [114]:
vgg_lr_flatting = keras.Model(inputs=clf1.input, outputs=clf1.get_layer(index=9).output)
vgg_featureVector_flatting = vgg_lr_flatting.predict(trainX)

30/30 [==============================] - 4s 88ms/step


In [115]:
vgg_featureVector_flatting.shape

(947, 64)

In [116]:
VGG_LR_FLATTING = GaussianNB().fit(vgg_featureVector_flatting, np.ravel(trainY))

In [117]:
vgg_featureVector_flatting_test = vgg_lr_flatting.predict(testX)

4/4 [==============================] - 0s 89ms/step


In [118]:
acc = VGG_LR_FLATTING.score(vgg_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 100.000


# MobileNet + Flatting + LogisticRegression

In [119]:
clf2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 32)          589856    
                                                                 
 batch_normalization_3 (Batc  (None, 4, 4, 32)         128       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 32)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 64)         

In [120]:
mobile_lr_flatting = keras.Model(inputs=clf2.input, outputs=clf2.get_layer(index=9).output)
mobile_featureVector_flatting = mobile_lr_flatting.predict(trainX)

30/30 [==============================] - 4s 89ms/step


In [121]:
mobile_featureVector_flatting.shape

(947, 64)

In [122]:
mobile_LR_FLATTING = RandomForestClassifier(n_estimators=100, random_state=1).fit(mobile_featureVector_flatting, np.ravel(trainY))

In [123]:
mobile_featureVector_flatting_test = mobile_lr_flatting.predict(testX)

4/4 [==============================] - 0s 92ms/step


In [124]:
acc = mobile_LR_FLATTING.score(mobile_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 100.000


# Resnet + Flatting + LogisticRegression

In [125]:
clf3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228864   
 nal)                                                            
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          294944    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                        

In [126]:
resnet_lr_flatting = keras.Model(inputs=clf3.input, outputs=clf3.get_layer(index=9).output)
resnet_featureVector_flatting = resnet_lr_flatting.predict(trainX)

30/30 [==============================] - 1s 35ms/step


In [127]:
resnet_featureVector_flatting.shape

(947, 64)

In [128]:
resnet_LR_FLATTING = LogisticRegression(random_state=0).fit(resnet_featureVector_flatting, np.ravel(trainY))

In [129]:
resnet_featureVector_flatting_test = resnet_lr_flatting.predict(testX)

4/4 [==============================] - 0s 35ms/step


In [130]:
acc = resnet_LR_FLATTING.score(resnet_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 100.000


# Resnet + Pooling + LogisticRegression

In [131]:
#new_model_lr_pooling = keras.Model(inputs=clf3.input, outputs=clf3.get_layer(index=8).output)
#featureVector_pooling = new_model_lr_pooling.predict(trainX)

In [132]:
#featureVector_pooling.shape

In [133]:
#LR_POOLING = LogisticRegression(random_state=0).fit(featureVector_pooling, np.ravel(trainY))

In [134]:
#featureVector_pooling_test = new_model_lr_pooling.predict(testX)

In [135]:
#acc = LR_POOLING.score(featureVector_pooling_test, testY)
#print('Acurácia: %.3f' % (acc * 100.0))

# Emsemble Gaussian + RandomForest + LogisticRegression (VGG , MobileNet, Resnet)

In [136]:
#creating the emsemble
eclf = VotingClassifier(estimators=[('lr_vgg', VGG_LR_FLATTING), ('gaussian_mobile', mobile_LR_FLATTING), ('rf_resnet', resnet_LR_FLATTING)], voting='soft')

In [137]:
#joining the features obtained by flatting on all models
emsemble_feature_vector = np.append(vgg_featureVector_flatting, mobile_featureVector_flatting, axis=1)
emsemble_feature_vector = np.append(emsemble_feature_vector, resnet_featureVector_flatting, axis=1)

In [138]:
emsemble_feature_vector.shape

(947, 192)

In [139]:
#training the emsemble with all features
eclf.fit(emsemble_feature_vector, np.ravel(trainY))

VotingClassifier(estimators=[('lr_vgg', GaussianNB()),
                             ('gaussian_mobile',
                              RandomForestClassifier(random_state=1)),
                             ('rf_resnet', LogisticRegression(random_state=0))],
                 voting='soft')

In [140]:
emsemble_feature_vector_test = np.append(vgg_featureVector_flatting_test, mobile_featureVector_flatting_test, axis=1)
emsemble_feature_vector_test = np.append(emsemble_feature_vector_test, resnet_featureVector_flatting_test, axis=1)

In [141]:
acc = eclf.score(emsemble_feature_vector_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 100.000
